In [ ]:
"""
Topic: IAM Roles & Permissions
==============================
Demonstrates how to create, attach, and test IAM roles and policies
programmatically using boto3 in AWS.
"""

import boto3
import json
from botocore.exceptions import ClientError

# -------------------------------------------------------------------
# Step 1 – Initialize IAM Client
# -------------------------------------------------------------------
iam = boto3.client('iam')
print("🔐 Initialized IAM client successfully.")

# -------------------------------------------------------------------
# Step 2 – Create a Policy
# -------------------------------------------------------------------
def create_policy(policy_name):
    """Creates a custom IAM policy with S3 read-only access."""
    policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": ["s3:GetObject", "s3:ListBucket"],
                "Resource": [
                    "arn:aws:s3:::supplychain-data-demo/trusted/*",
                    "arn:aws:s3:::supplychain-data-demo"
                ]
            }
        ]
    }

    try:
        response = iam.create_policy(
            PolicyName=policy_name,
            PolicyDocument=json.dumps(policy_document)
        )
        print(f"✅ Policy '{policy_name}' created successfully.")
        return response["Policy"]["Arn"]
    except ClientError as e:
        print(f"⚠️ Policy creation skipped or failed: {e}")
        return None

# -------------------------------------------------------------------
# Step 3 – Create an IAM Role
# -------------------------------------------------------------------
def create_role(role_name):
    """Creates a new IAM role with a trust relationship for Glue."""
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {"Service": "glue.amazonaws.com"},
                "Action": "sts:AssumeRole"
            }
        ]
    }
    try:
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy)
        )
        print(f"✅ Role '{role_name}' created successfully.")
        return response["Role"]["Arn"]
    except ClientError as e:
        print(f"⚠️ Role creation failed or already exists: {e}")
        return None

# -------------------------------------------------------------------
# Step 4 – Attach Policy to Role
# -------------------------------------------------------------------
def attach_policy_to_role(role_name, policy_arn):
    """Attach a custom policy to a specific IAM role."""
    try:
        iam.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
        print(f"🔗 Attached policy to role '{role_name}' successfully.")
    except ClientError as e:
        print(f"❌ Failed to attach policy: {e}")

# -------------------------------------------------------------------
# Step 5 – List Roles and Policies
# -------------------------------------------------------------------
def list_iam_entities():
    print("\n📋 Listing IAM Roles and Custom Policies:")
    roles = iam.list_roles()["Roles"]
    for r in roles[:3]:  # show a few roles
        print(f" - {r['RoleName']}")
    policies = iam.list_policies(Scope="Local")["Policies"]
    for p in policies[:3]:
        print(f" - {p['PolicyName']}")

# -------------------------------------------------------------------
# Step 6 – Main Flow
# -------------------------------------------------------------------
def main():
    print("\n🚀 IAM Roles & Permissions Demo Start\n")
    role_name = "GlueETLRoleDemo"
    policy_name = "GlueETLReadPolicyDemo"

    policy_arn = create_policy(policy_name)
    role_arn = create_role(role_name)
    if policy_arn:
        attach_policy_to_role(role_name, policy_arn)
    list_iam_entities()
    print("\n✅ IAM setup completed successfully!")

if __name__ == "__main__":
    main()
